## LangGraph Documentation
* [LangGraph Documentation](https://python.langchain.com/docs/langgraph/)

## Overview
* Framework to develop Multi-Agent LLM Apps.
* Python and Javascript versions.
* Main uses:
    * add cycles to LLM App.
    * add persistence to LLM App.

In [2]:
#!pip install -U langgraph

## Terminology
* Graph: agent, multi-agent.
* Nodes: actions.
* Edges: node connections.
* State: memory.

## Basic agent

In [3]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

* First, we initialize our model and an agent.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

agent = MessageGraph()

* Next, we add a single node to the agent, called "node1", which simply calls the LLM with the given input.

In [5]:
agent.add_node("node1", llm)

* We add an edge from this "node1" node to the special string END (`__end__`). This means that execution will end after the current node.

In [6]:
agent.add_edge("node1", END)

* We set "node1" as the entry point to the agent.

In [7]:
agent.set_entry_point("node1")

* We compile the agent, translating it to low-level pregel operations ensuring that it can be run.

In [8]:
runnable_agent = agent.compile()

* We can run the agent now:

In [9]:
runnable_agent.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', additional_kwargs={}, response_metadata={}, id='66512616-6f68-40b8-a751-ac7035e0198d'),
 AIMessage(content='1 + 1 equals 2.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 15, 'total_tokens': 24, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4fbe98c1-d424-4db7-8cdc-4ee607e4b4ed-0', usage_metadata={'input_tokens': 15, 'output_tokens': 9, 'total_tokens': 24})]

When we execute the agent:
* LangGraph adds the input message to the  state, then passes the state to the entrypoint node, "node1".
* The "node1" node executes, invoking the chat model.
* The chat model returns an AIMessage. LangGraph adds this to the state.
* Execution progresses to the special END value and outputs the final state.
* And as a result, we get a list of two chat messages as output.

## Agent with a router and conditional edges
* Let's allow the LLM to conditionally call a "multiply" node using tool calling.
* We'll recreate our previous agent with an additional "multiply" tool that will take the result of the most recent message, if it is a tool call, and calculate the result.
* We will bind the multiply tool to the OpenAI model to allow the llm to optionally use the tool.

In [10]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

llm = ChatOpenAI(temperature=0)
llm_with_tools = llm.bind_tools([multiply])

In [11]:
agent_with_conditional_edges = MessageGraph()
agent_with_conditional_edges.add_node("node1", llm_with_tools)
agent_with_conditional_edges.set_entry_point("node1")

Let's create the second node:

In [12]:
tool_node = ToolNode([multiply])
agent_with_conditional_edges.add_node("multiply", tool_node)

Let's create the edge:

In [13]:
agent_with_conditional_edges.add_edge("multiply", END)

Using conditional edges, which call a function on the current state and routes execution to a node the function's output:
* If the "node1" node returns a message expecting a tool call, we want to execute the "multiply" node.
* If not, we can just end execution.

In [14]:
from typing import Literal, List
from langchain_core.messages import BaseMessage

def router(state: List[BaseMessage]) -> Literal['multiply', '__end__']:
    tool_calls = state[-1].additional_kwargs.get('tool_calls', [])
    if len(tool_calls):
        return 'multiply'
    else:
        return '__end__'
    
agent_with_conditional_edges.add_conditional_edges('node1', router)


In [15]:
runnable_agent_with_conditional_edges = agent_with_conditional_edges.compile()

In [16]:
runnable_agent_with_conditional_edges.invoke(HumanMessage("What is 123 * 456?"))

[HumanMessage(content='What is 123 * 456?', additional_kwargs={}, response_metadata={}, id='f6bf203d-4b77-4e80-8666-fc2dbe601f34'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Acj5HMfTxE2BreUFPDHDTe1q', 'function': {'arguments': '{"first_number": 123, "second_number": 456}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 57, 'total_tokens': 92, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c541656f-e415-4e97-8b17-c53395c6cf36-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_Acj5HMfTxE2BreUFPDHDTe1q', 'type': 'tool_call'}], usage_metadata={'in

While conversational responses are outputted directly:

In [17]:
runnable_agent_with_conditional_edges.invoke(HumanMessage('What is your name?'))

[HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={}, id='812ec503-2f0c-4629-8b6a-322d3f56a3b1'),
 AIMessage(content='My name is Assistant. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 54, 'total_tokens': 68, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3862a95b-6326-47b3-8f3a-045e454422dd-0', usage_metadata={'input_tokens': 54, 'output_tokens': 14, 'total_tokens': 68})]

## Agent with cycles

* We will use Tavily as online search tool:


In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults

online_search_tool = [TavilySearchResults(max_results=1)]

* We can now wrap these tools in a simple LangGraph ToolNode. This class receives the list of messages (containing tool_calls), calls the tool(s) the LLM has requested to run, and returns the output as new ToolMessage(s).

In [19]:
tool_node_with_online_search = ToolNode(online_search_tool)

* Let's make sure the model knows that it has these tools available to call. We can do this by converting the LangChain tools into the format for OpenAI tool calling using the bind_tools() method.

In [20]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, streaming=True)
llm_with_online_search_tool = llm.bind_tools(online_search_tool)

* Let's now define the state of the agent.
* Each node will return operations to update the state.
* For this example, we want each node to just add messages to the state. Therefore, in this case the state of the agent will be a list of messages. In other projects, the state can be any type.
* We will use a TypedDict with one key (messages) and annotate it so that we always add to the messages key when updating it using the `is always added to` with the second parameter (operator.add).

In [21]:
from typing import TypedDict, Annotated

def add_messages(left:list, right:list):
    """Add-don´t overwrite."""
    return left + right

class AgenState(TypedDict):
    #The 'add_messages' function within the annotation defines
    # *how* updates should be merged into the state
    messages: Annotated[list, add_messages]
    

* Let's now define the nodes of this agent:
    * the node responsible for deciding what (if any) actions to take.
    * if the previous node decides to take an action, this second node will then execute that action calling the online search tool.
    

* We will also define the edges that will interconnect the nodes.
* One will be a Conditional Edge. After the node1 is called, the llm will dedice either:
    * a. Run the online search tool (node2), OR
    * b. Finish

* The second will be a normal Edge: after the online search tool (node2) is invoked, the graph should always return to the node1 to decide what to do next.

In [22]:
from typing import Literal

def should_continue(state: AgenState) -> Literal['node2', '__end__']:
    messages = state['messages']
    last_message = messages[-1]
    #If the LLM makes a tool call, then we route to the action node
    if last_message.tool_calls:
        return 'node2'
    else:
    # Otherwise, we stop (reply to the user)
        return '__end__'
    
#Define the function that calls the model
def call_model(state: AgenState):
    messages = state['messages']
    response = llm_with_online_search_tool.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}
    


In [26]:
from langgraph.graph import StateGraph, END
# Define a new graph (agent)
workflow = StateGraph(AgenState)

# Define the two nodes we will cycle between
workflow.add_node("node1", call_model)
workflow.add_node("node2", tool_node_with_online_search)

# Set the entrypoint as 'node1
# This means that this node is the first one called
workflow.set_entry_point('node1')

workflow.add_conditional_edges(
    # First, we define the start node. We use `node1`.
    # This means these are the edges taken after the `node1` node is called.
    "node1",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `online_search_tool` to `node1`.
# This means that after `online_search_tool` is called, `node1` node is called next.
workflow.add_edge('node2', 'node1')


In [27]:
agent_with_cycles = workflow.compile()

In [28]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
agent_with_cycles.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_SrFwhyyQESTI5CGmlm9hoTaB', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-d1bb0371-9f4e-41f9-935f-763145368862-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_SrFwhyyQESTI5CGmlm9hoTaB', 'type': 'tool_call'}]),
  ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1735645505, \'localtime\': \'2024-12-31 03:45\'}, \'current\': {\'last_

## Additional documentation
* [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/).
* [LangGraph How-To Guides](https://langchain-ai.github.io/langgraph/how-tos/).